In [ ]:
from riotwatcher import LolWatcher, Handlers
import requests

#initialize user by SummonerName
lol_watcher = LolWatcher('RGAPI-65b11669-2428-4123-a4b6-192f9e2e504b')
my_region_old = "euw1"
my_region_new = "europe"
player = lol_watcher.summoner.by_name(my_region_old, 'Don Arts')

In [ ]:
latest = lol_watcher.match.matchlist_by_puuid(my_region_new, player['puuid'], count=1)
match_id_split = latest[0].split(sep="_")
match_id_int = int(match_id_split[1])

In [27]:
matchlist = []
#Riot API is limited to 100 requests/min
for i in range(100):
    try:
        matchlist.append(lol_watcher.match.by_id(my_region_new, "EUW1_"+str(match_id_int-i)))
    except requests.HTTPError as e:
        print("errrr")
    print(str(match_id_int-i) + "found and added")
    print(str(i)+"/100")

5760922292found and added
0/100
5760922291found and added
1/100
5760922290found and added
2/100
5760922289found and added
3/100
errrrrr
5760922288found and added
4/100
5760922287found and added
5/100
errrrrr
5760922286found and added
6/100
errrrrr
5760922285found and added
7/100
errrrrr
5760922284found and added
8/100
5760922283found and added
9/100
errrrrr
5760922282found and added
10/100
errrrrr
5760922281found and added
11/100
5760922280found and added
12/100
5760922279found and added
13/100
5760922278found and added
14/100
5760922277found and added
15/100
5760922276found and added
16/100
5760922275found and added
17/100
errrrrr
5760922274found and added
18/100
errrrrr
5760922273found and added
19/100
5760922272found and added
20/100
5760922271found and added
21/100
5760922270found and added
22/100
5760922269found and added
23/100
5760922268found and added
24/100
5760922267found and added
25/100
5760922266found and added
26/100
5760922265found and added
27/100
5760922264found and ad